In [1]:
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
# from qcodes.plots.pyqtgraph import QtPlot
from qcodes.loops import Loop
from time import sleep
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json
import os
import sys


In [2]:
sys.path.append('C:\QCodes_Driver_And_Scripts\Drivers')

from qcodes.instrument_drivers.yokogawa.GS200 import GS200

from RS_SMB_100A import SMB100A
from RS_SGS_100A import SGS100A
from HP3245A import HP3245A
from RS_FSQ import RS_FSQ
from VNA_RS import VNA
      
try:

    twpa_source=SGS100A('twpa_pump','TCPIP0::192.168.10.56::inst0::INSTR')

except KeyError as er:

    SMB100A.close_all()
    twpa_source=SGS100A('twpa_pump','TCPIP0::192.168.10.56::inst0::INSTR')
     

try:
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
               init_s_params=False,terminator='\n')

except KeyError as er:


    VNA.close_all() #Disconnect and irreversibly tear down the instrument
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
               init_s_params=False,terminator='\n')
    
try:
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")


except KeyError as er:

    GS200.close_all() #Disconnect and irreversibly tear down the instrument
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")

Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/114045, firmware:4.2.76.0-4.30.046.295) in 0.65s
Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062101578, firmware:2.80) in 0.02s
Connected to: YOKOGAWA GS210 (serial:90Y438440, firmware:2.02) in 0.02s


In [3]:

def measure_vs_current(current_vec, exp, station):


    VNA.rf_on()

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(I_source.current)
    meas.register_parameter(VNA.channels.S21.trace_mag_phase,
                            setpoints=[I_source.current])


    meas.write_period = 10

    with meas.run() as datasaver:

        id=datasaver.dataset.run_id
        qc.load_by_run_spec( captured_run_id=id).add_metadata('parameter_snap',
                             json.dumps(parameter_snap))

        for current in tqdm(current_vec):

            I_source.current(current)

            get_v = VNA.channels.S21.trace_mag_phase.get()
            get_i = I_source.current.get()

            datasaver.add_result((VNA.channels.S21.trace_mag_phase, get_v),(I_source.current,get_i))

In [4]:
start_all_logging()

# Create a station
station = qc.Station()
station.add_component(VNA)
station.add_component(I_source)
station.add_component(twpa_source)

station.snapshot()
station.components

# Experiment details
date=datetime.datetime.today().strftime('%Y_%m_%d')
user='DF'
description='VNA_vs_current'
database_name = date+"_"+user+"_"+description

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\230113-18256-qcodes.log


2023-01-13 14:25:20,101 ¦ qcodes.instrument.instrument_base ¦ WARNING ¦ instrument_base ¦ snapshot_base ¦ 292 ¦ [gs200_program(GS200Program)] Snapshot: Could not update parameter: trigger


In [5]:
script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

In [6]:
qc.initialise_or_create_database_at(data_dir+'\\'+database_name+'.db')
qc.config.core.db_location

parameter_snap={}

In [7]:
I_start  =  -2.85
I_end    =  -2.6
# I_step   = 0.0022

# points_I =  int(np.abs(I_end-I_start)/I_step)
points_I = 100

I_vec    = np.linspace(I_start, I_end, points_I)*1.e-3

# Snapshot parameters
parameter_snap['flux']={'I_start':I_vec[0]*1.e3,'I_end':I_vec[-1]*1.e3,'I_points':len(I_vec)}

In [8]:
I_step   = np.abs(I_end-I_start)/points_I

In [9]:
I_step

0.0025

In [10]:
points_I

100

In [11]:
# TWPA PUMP PARAMETERS
pump_freq    = 4.998                #GHz
pump_power   = 0.5                #dBm
pump_status  = 'on'
twpa_current = -0.5               #mA

print('\nMeasurement with TWPA {}'.format(pump_status))


Measurement with TWPA on


In [12]:
start_freq = 2.6
end_freq   = 3.7
VNA_power    = -20
IF_bandwidth = 30
Averages     = 1
points_VNA   = 1001

trace_time   = points_VNA / IF_bandwidth

print('\ntime for one trace : {}'.format(datetime.timedelta(seconds=trace_time)))
print('\ntime estimation for all measurement : {} '
      .format(datetime.timedelta(seconds=Averages*len(I_vec)*trace_time)))

# Snapshot parameters
parameter_snap['vna'] = {'start_freq':start_freq, 'end_freq':end_freq,
                         'VNA_power':VNA_power, 'IF_bandwidth':IF_bandwidth,
                         'Averages':Averages, 'points':points_VNA}


time for one trace : 0:00:33.366667

time estimation for all measurement : 0:55:36.666667 


In [13]:
VNA.add_channel('S21',vna_parameter='S21')

VNA.channels.S21.format('dB')
VNA.channels.S21.bandwidth(IF_bandwidth)
VNA.channels.S21.power(VNA_power)
VNA.channels.S21.avg(Averages)
VNA.channels.npts(points_VNA)
VNA.channels.start(start_freq*1e9)
VNA.channels.stop(end_freq*1e9)

2023-01-13 14:25:20,297 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\instrument_base.py:143: QCoDeSDeprecationWarning: Parameter trace_mag_phase on instrument ZNB20_S21 does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(

2023-01-13 14:25:20,303 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\instrument_base.py:143: QCoDeSDeprecationWarning: Parameter trace on instrument ZNB20_S21 does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(

2023-01-13 14:25:20,309 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\instrument_base.py:143: QCoDeSDeprecati

In [14]:
if I_source.output()=='on':
    I_source.output('off')

I_source.source_mode('CURR')
I_source.range(10e-3)

I_source.current(I_vec[0])

I_source.output('on')

twpa_source.frequency(pump_freq*1.e9)
twpa_source.power(pump_power)
twpa_source.status(pump_status.lower())

In [15]:
# sleep(3600)

In [16]:
# Load experiment
exp_name = 'VNA_vs_current'

# sample_name = 'RKW0310'
sample_name = 'B5_G3_GALV_SQ200_1ST'

exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample_name)

measure_vs_current(I_vec, exp=exp, station=station)

Starting experimental run with id: 8. 


  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
# Shutdown devices
VNA.rf_off()
VNA.cont_meas_off()
I_source.output('off')
twpa_source.status('off')